# カーネルを使った強化学習

参考：
* [Kernel-Based Reinforcement Learning: A Finite-Time Analysis](https://arxiv.org/abs/2004.05599)

状態行動空間が連続のMDPではどのようにRLすればよいでしょうか？
今回はカーネルを使った強化学習を見てみます．

表記：
* $(x, a)$に定義された測度$P$と任意の関数$f$について，$P f(x, a)=P(\cdot \mid x, a) f=\int f(y) \mathrm{d} P(y \mid x, a)$
* $\mathcal{X}$と$\mathcal{A}$上に定義された測度：$\rho:(\mathcal{X} \times \mathcal{A})^2 \rightarrow \mathbb{R}_{\geq 0}$
* $\mathcal{X}$上の測度空間：$\left(\mathcal{X}, \mathcal{T}_{\mathcal{X}}\right)$．$\mathcal{T}_{\mathcal{X}}$はσ代数です．
* 報酬関数：$\left\{r_h\right\}_{h \in[H]}$は$\mathcal{X} \times \mathcal{A}$ to $[0,1]$の集合
* 遷移カーネル：$P=\left\{P_h\right\}_{h \in[H]}$．ここで，$P_h$は$(\mathcal{X} \times \mathcal{A}) \times \mathcal{T}_{\mathcal{X}}$ to $\mathbb{R}_{\geq 0}$なるカーネル．
* ベルマン方程式：$Q_h^*(x, a) \stackrel{\text { def }}{=} r_h(x, a)+\int_{\mathcal{X}} V_{h+1}^*(y) \mathrm{d} P_h(y \mid x, a)$
* リグレット：$\mathcal{R}(K) \stackrel{\text { def }}{=} \sum_{k=1}^K\left(V_1^*\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right)$


仮定：

---

* $(\mathcal{X}\times \mathcal{A})\times (\mathcal{X}\times \mathcal{A})$上の測度$\rho$は既知であるとする．
* 任意の$\left(x, x^{\prime}, a, a^{\prime}\right)$について，
$$
\rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]=\rho_{\mathcal{X}}\left(x, x^{\prime}\right)+\rho_{\mathcal{A}}\left(a, a^{\prime}\right)
$$
が成り立つような測度$\rho_{\mathcal{X}}$と$\rho_{\mathcal{A}}$が存在する．

---

報酬と遷移は次の意味でリプシッツ連続ですL

$$
\left|r_h(x, a)-r_h\left(x^{\prime}, a^{\prime}\right)\right| \leq \lambda_r \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

および

$$
W_1\left(P_h(\cdot \mid x, a), P_h\left(\cdot \mid x^{\prime}, a^{\prime}\right)\right) \leq \lambda_p \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

ここで，$W_1$は1-Wasserstein距離です．つまり，$W_1(\mu, \nu) \stackrel{\text { def }}{=}\sup _{f: \operatorname{Lip}(f) \leq 1} \int_{\mathcal{X}} f(y)(\mathrm{d} \mu(y)-\mathrm{d} \nu(y))$

---

このようなリプシッツ連続な報酬と遷移のもとでは最適Q値がリプシッツ連続になります．

---

**補題**

$L_h \stackrel{\text { def }}{=} \sum_{h^{\prime}=h}^H \lambda_r \lambda_p^{H-h^{\prime}}$
とする．このとき，

$$
\left|Q_h^*(x, a)-Q_h^*\left(x^{\prime}, a^{\prime}\right)\right| \leq L_h \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

が成立します．証明は簡単なので省略．

---



## カーネルUCBVI

$u, v \in \mathcal{X} \times \mathcal{A}$とします．関数$g: \mathbb{R}_{\geq 0} \rightarrow[0,1]$について，カーネル関数を

$$\psi_\sigma(u, v) \stackrel{\text { def }}{=} g(\rho[u, v] / \sigma)$$

として定めます．

---

**仮定**

関数$g: \mathbb{R}_{\geq 0} \rightarrow[0,1]$は微分可能，non-increasing，$g(4) > 0$を満たし，また，次を満たす定数$C^g_1, C^g_2$が存在します：

$$
g(z) \leq C_1^g \exp \left(-z^2 / 2\right) \text { and } \sup _z\left|g^{\prime}(z)\right| \leq C_2^g
$$

例えば$g(z)=\exp \left(-z^2 / 2\right)$のようなガウスカーネルが仮定を満たしています．

---

カーネルUCBVIではカーネル関数を使って過去のデータを重み付けし，報酬や遷移確率を推定します．

$(x, a)$ and $(s, h) \in[K] \times[H]$に対して，重み関数を

$$
\begin{aligned}
& w_h^s(x, a) \stackrel{\text { def }}{=} \psi_\sigma\left((x, a),\left(x_h^s, a_h^s\right)\right) \quad \text { and } \\
& \widetilde{w}_h^s(x, a) \stackrel{\text { def }}{=} \frac{w_h^s(x, a)}{\beta+\sum_{l=1}^{k-1} w_h^l(x, a)},
\end{aligned}
$$

とします．ここで$\beta > 0$は正則化項です．この重みを使って，

$$
\begin{aligned}
& \widehat{r}_h^k(x, a) \stackrel{\text { def }}{=} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) r_h^s, \\
& \widehat{P}_h^k(y \mid x, a) \stackrel{\text { def }}{=} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) \delta_{x_{h+1}^s}(y) .
\end{aligned}
$$

のように報酬と遷移を計算します．
そして，推定した報酬と遷移を使って

$$
\widetilde{Q}_h^k(x, a)=\widehat{r}_h^k(x, a)+\widehat{P}_h^k V_{h+1}^k(x, a)+\mathrm{B}_h^k(x, a)
$$

のようにQ値を計算します．ここで$B$は探索用のボーナスです．
これを使って，

$$
Q_h^k(x, a) \stackrel{\text { def }}{=} \min _{s \in[k-1]}\left(\widetilde{Q}_h^k\left(x_h^s, a_h^s\right)+L_h \rho\left[(x, a),\left(x_h^s, a_h^s\right)\right]\right)
$$

のようにしてQ値の間隔を補完します．

---

**ボーナスについて**


$$
\mathbf{C}_h^k(x, a) \stackrel{\text { def }}{=} \beta+\sum_{s=1}^{k-1} w_h^s(x, a)
$$

を訪問回数の一般化とします．これを使って，ボーナスを

$$
\mathrm{B}_h^k(x, a) \approx \frac{H}{\sqrt{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+L_1 \sigma
$$

として定義します．

以上をまとめて，Kernel-UCBVIは次の処理を行います：

![KUCBVI](figs/Kernel-UCBVI-OptimisticQ.png)
![KUCBVI](figs/Kernel-UCBVI.png)

